# Converting a List of DICOMs to PNG

In [1]:
import pydicom
import numpy as np
import pandas as pd
import png

In [8]:
# Get DICOM image metadata
class DCM_Tags():
    def __init__(self, img_dcm):
        try:
            self.laterality = img_dcm.ImageLaterality
        except AttributeError:
            self.laterality = np.nan
            
        try:
            self.view = img_dcm.ViewPosition
        except AttributeError:
            self.view = np.nan
            
        try:
            self.orientation = img_dcm.PatientOrientation
        except AttributeError:
            self.orientation = np.nan

# Check whether DICOM should be flipped
def check_dcm(imgdcm):
    # Get DICOM metadata
    tags = DCM_Tags(imgdcm)
    
    # If image orientation tag is defined
    if ~pd.isnull(tags.orientation):
        # CC view
        if tags.view == 'CC':
            if tags.orientation[0] == 'P':
                flipHorz = True
            else:
                flipHorz = False
            
            if (tags.laterality == 'L') & (tags.orientation[1] == 'L'):
                flipVert = True
            elif (tags.laterality == 'R') & (tags.orientation[1] == 'R'):
                flipVert = True
            else:
                flipVert = False
        
        # MLO or ML views
        elif (tags.view == 'MLO') | (tags.view == 'ML'):
            if tags.orientation[0] == 'P':
                flipHorz = True
            else:
                flipHorz = False
            
            if (tags.laterality == 'L') & ((tags.orientation[1] == 'H') | (tags.orientation[1] == 'HL')):
                flipVert = True
            elif (tags.laterality == 'R') & ((tags.orientation[1] == 'H') | (tags.orientation[1] == 'HR')):
                flipVert = True
            else:
                flipVert = False
        
        # Unrecognized view
        else:
            flipHorz = False
            flipVert = False
            
    # If image orientation tag is undefined
    else:
        # Flip RCC, RML, and RMLO images
        if (tags.laterality == 'R') & ((tags.view == 'CC') | (tags.view == 'ML') | (tags.view == 'MLO')):
            flipHorz = True
            flipVert = False
        else:
            flipHorz = False
            flipVert = False
            
    return flipHorz, flipVert

# Save DICOM pixel array as PNG
def save_dcm_image_as_png(image, png_filename, bitdepth=12):
    with open(png_filename, 'wb') as f:
        writer = png.Writer(height=image.shape[0], 
                            width=image.shape[1], 
                            bitdepth=bitdepth, 
                            greyscale=True)
        writer.write(f, image.tolist())

In [10]:
# Convert list of DICOMs to PNGs
def process_dcm_list(dcm_list, save_path):
    for i, dcm_path in enumerate(dcm_list):
        print(f"Processing DICOM #{i}...")
        
        # Load DICOM
        dcm = pydicom.dcmread(dcm_path)
        img = dcm.pixel_array
        
        # Check if a horizontal flip is necessary
        horz, _ = check_dcm(dcm)
        if horz:
            # Flip img horizontally
            img = np.fliplr(img)
        
        # Get new file name
        split_fn = dcm_path[:-4].split('/')
        new_fn = f"{split_fn[-1]}_conv.png"
        
        # Save PNG
        png_path = save_path + new_fn
        save_dcm_image_as_png(img, png_path)

In [11]:
# Provide a list of DICOM paths and a target directory
dcm_list = []
save_path = "/PATH/TO/SAVE/DIR/"

# Convert DICOMs
process_dcm_list(dcm_list, save_path)

Processing DICOM #0...
Processing DICOM #1...
Processing DICOM #2...
Processing DICOM #3...
Processing DICOM #4...
Processing DICOM #5...
Processing DICOM #6...
Processing DICOM #7...
Processing DICOM #8...
Processing DICOM #9...
